In [72]:
import numpy as np
import pandas as pd

In [73]:
ts = pd.read_csv('../data/tritonsearch_results.csv')
ucs = pd.read_csv('../data/ucs_results_labeled.csv')

test_results = [ts, ucs]

In [74]:
ts.head(10)

,Query,Class Code,Ranking (position),Relevance
0,2d art,NO RESULTS,1,0
1,2d art,NO RESULTS,2,0
2,2d art,NO RESULTS,3,0
3,2d art,NO RESULTS,4,0
4,2d art,NO RESULTS,5,0
5,accounting,MGT 45,1,1
6,accounting,MGT 138,2,1
7,accounting,MGTP 422,3,1
8,accounting,MGT 143,4,1
9,accounting,MGT 147,5,1


In [75]:
ucs.head(10)

,Query,Class Code,Ranking (position),Relevance
0,2d art,VIS 1,1,1
1,2d art,VIS 106A,2,1
2,2d art,VIS 2,3,1
3,2d art,VIS 105A,4,1
4,2d art,VIS 3,5,0
5,accounting,MGT 404,1,1
6,accounting,MGTF 401,2,1
7,accounting,MGT 4,3,1
8,accounting,ECON 4,4,1
9,accounting,MGT 138,5,1


## Mean Reciprocal Rank

In [79]:
# MRR
mrrs = []

for df in test_results:

    # relevant_results = df[df['Relevance'] == 1]

    # # Get the minimum rank for each query, assuming relevance of 1 means a relevant result
    # min_rank_per_query = relevant_results.groupby('Query')['Ranking (position)'].min()

    # # Calculate reciprocal rank
    # reciprocal_rank = 1 / min_rank_per_query

    # # Calculate MRR
    # mrr = reciprocal_rank.mean()

    # -----------------------------------------------------------------------------------------------------------------------------------

#     relevant_results_new = df[df['Relevance'] == 1]
    # If a query has no relevant results (i.e., all relevances are 0), we set its minimum rank to negative infinity for MRR calculation purposes
    # This is because MRR is undefined (or considered as 0) for queries with no relevant results
    df['Reciprocal Rank'] = df.apply(lambda x: 1/x['Ranking (position)'] if x['Relevance'] == 1 else float('-inf'), axis=1)
    max_reciprocal_rank_per_query = df.groupby('Query')['Reciprocal Rank'].max()

    # Handle cases where all results are irrelevant by setting their reciprocal rank contribution to 0
    max_reciprocal_rank_per_query = max_reciprocal_rank_per_query.replace(float('-inf'), 0)

    # Calculate MRR
    mrr = max_reciprocal_rank_per_query.mean()

    mrrs.append(mrr)

mrrs

[0.7241379310344828, 0.9689655172413794]

## NDCG

In [78]:
# Function to calculate DCG
def dcg(scores):
    return np.sum(
        (2**scores - 1) / np.log2(np.arange(2, scores.size + 2))
    )

# Function to calculate NDCG for each query
def ndcg(df):
    df_sorted = df.sort_values(by='Ranking (position)')
    actual_relevances = df_sorted['Relevance'].values
    ideal_relevances = np.sort(df_sorted['Relevance'].values)[::-1]
    
    dcg_actual = dcg(actual_relevances)
    dcg_ideal = dcg(ideal_relevances)
    
    return dcg_actual / dcg_ideal if dcg_ideal > 0 else 0

# Calculate NDCG for each query
ndcg_list = []
for df in test_results:
    ndcg_scores = df.groupby('Query').apply(ndcg, include_groups=False) 
    ndcg_list.append(ndcg_scores.mean())

ndcg_list

[0.7265044669826709, 0.9684426357451889]